In [ ]:
import json

import requests
from flask import Flask
from flask import request, render_template

from paper.utils import post_request
from paper.config import *


app = Flask(__name__)

@app.route("/")
def hello():
    """Render user interface with search form from templates"""
    return render_template("index.html")


@app.route(SEARCH_PATH, methods=['GET', 'POST'])
def search():
    """
    Search server provide UI hosting and all search workflow
    Receive ajax post request from user interface with meta and search  
    """
    if request.method=='POST':    
        data = request.json
        meta = data['meta']
        query = data['query']
        
        analyze_json = {'textes' : [query]}
        r_analyzing_query = post_request(ANALYZER_PORT, ANALYZER_PATH, analyze_json)
        stemmed_query = r_analyzing_query['stemmed_docs'][0]
        query_lang = r_analyzing_query['langs'][0]

        query_json = {"stemmed_query": stemmed_query, "meta":"lalka"}  
        r_rank = post_request(RANK_PORT, RANK_PATH, query_json)
        docs = r_rank["ranked"]

        return json.dumps({'status': "ok", "results": docs})
    else:
        return "give me some text by post json {text: 'your text here'}"
    

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=SEARCH_PORT)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:13568/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Nov/2018 17:14:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2018 17:14:28] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2018 17:14:39] "POST /search HTTP/1.1" 200 -
